### Grupo 10
Miguel Robledo Casal  
Gonzalo Figueroa del Val  
Guillermo Calvo Suarez  

# Sokoban
Sokoban es un juego de tablero NxM en el que se deben empujar unas cajas a unas metas. Dado que la selección de N y M es arbitraria, existen infinitas configuraciones de paredes, cajas y metas. Aunque no todas las configuraciones tienen solución, asumiremos que todas las que le demos al agente son resolubles.  
Como el agente puede moverse en una de cuatro direcciones en cada paso, el espacio de estados crece exponencialmente. Podemos reducir el espacio de estados eliminando movimientos inútiles como caminar contra una pared y eliminando los estados en los que una caja se encuentre en una casilla muerta. Una casilla muerta es aquella desde la cual no se puede empujar una caja a una meta.  

La clase `Board` contiene métodos auxiliares para analizar y modificar un estado incluyendo el computo de casillas muertas, comprobación de si está resuelto o se puede avanzar en una dirección, etc.

In [1]:
class Board:
    def __init__(self, board, boxes, goals, player):
        self.board = board
        self.boxes = boxes
        self.goals = goals
        self.player = player
    
    def parse(str):
        rows = str.split("\n")
        
        board = []
        boxes = []
        goals = []
        player = (0, 0)
        for y, row in enumerate(rows):
            board.append([])
            
            for x, cell in enumerate(row):
                if cell in " -$@":
                    board[y].append(0)
                elif cell in "#":
                    board[y].append(1)
                elif cell in ".*+":
                    board[y].append(2)
                    goals.append((x, y))
                elif cell in "_!&":
                    board[y].append(3)
                
                if cell in "$*!":
                    boxes.append((x, y))
                if cell in "@+&":
                    player = (x, y)
        
        return Board(board, boxes, goals, player)
    
    def is_solved(self):
        return all(map(lambda x: self.__is_goal(*x), self.boxes))
    
    def is_deadlocked(self):
        return any(map(lambda x: self.board[x[1]][x[0]] == 3, self.boxes))
    
    def can_move_towards(self, direction):
        return self.__can_move_to(*self.__translate_towards(self.player, direction))
    
    def move_towards(self, direction):
        return self.__move_to(*self.__translate_towards(self.player, direction))
    
    def compute_dead(self):
        for y in range(len(self.board)):
            for x in range(len(self.board[y])):
                if self.board[y][x] == 0:
                    self.board[y][x] = 3
                elif self.board[y][x] == 2:
                    self.board[y][x] = 4
        
        def flood(x, y):
            if self.board[y][x] == 3:
                self.board[y][x] = 0
            elif self.board[y][x] == 4:
                self.board[y][x] = 2
            else:
                return
            
            for d in filter(lambda d: self.__can_pull_towards(x, y, d), "NESW"):
                print
                flood(*self.__translate_towards((x, y), d))
        
        for g in self.goals:
            flood(*g)
        
        return self
    
    def __translate_towards(self, coords, direction):
        return (coords[0] + (1 if direction == "E" else -1 if direction == "W" else 0),
                coords[1] + (1 if direction == "S" else -1 if direction == "N" else 0))
    
    def __is_inbounds(self, x, y):
        return y < len(self.board) and x < len(self.board[y])
    
    def __has_box(self, x, y):
        return (x, y) in self.boxes
    
    def __is_wall(self, x, y):
        return not self.__is_inbounds(x, y) or self.board[y][x] == 1
    
    def __is_goal(self, x, y):
        return self.board[y][x] == 2
    
    def __can_push_to(self, x, y):
        return not (self.__has_box(x, y) or self.__is_wall(x, y))
    
    def __can_move_to(self, x, y):
        return not self.__is_wall(x, y) and (not self.__has_box(x, y) or
                                           self.__can_push_to(2 * x - self.player[0], 2 * y - self.player[1]))
    
    def __move_to(self, x, y):
        if (x, y) in self.boxes:
            self.boxes[self.boxes.index((x, y))] = (2 * x - self.player[0], 2 * y - self.player[1])
        self.player = (x, y)
        
        return self
    
    def __can_pull_towards(self, x, y, direction):
        pos1 = self.__translate_towards((x, y), direction)
        pos2 = self.__translate_towards(pos1, direction)
        
        return not (self.__is_wall(*pos1) or self.__is_wall(*pos2))
    
    def __str__(self):
        ret = ""
        
        def select_char(x, y, char, box_char, player_char):
            return (box_char if (x, y) in self.boxes else
                    player_char if self.player == (x, y) else char)
        
        for y, row in enumerate(self.board):
            for x, cell in enumerate(row):
                if cell == 0:
                    ret += select_char(x, y, " ", "$", "@")
                elif cell == 1:
                    ret += "#"
                elif cell == 2:
                    ret += select_char(x, y, ".", "*", "+")
                elif cell == 3:
                    ret += select_char(x, y, "_", "!", "&")
            if y < len(self.board) - 1:
                ret += "\n"
        return ret

### Generación de mapas
Para verificar que la clase auxiliar lee correctamente los mapas, se generan 100 mapas de 100 caracteres de largo, se coloca un símbolo para el jugador, se le da la representación textual a la clase auxiliar, se convierte de vuelta a texto y se comprueba que las dos representaciones textuales son iguales.

In [2]:
import random

def test():
    chars = " #$.*\n"
    board = "".join([random.choice(chars) for _ in range(100)])
    player = random.randrange(len(board))
    board = board[:player] + ("+" if board[player] == "." else "@") + board[player+1:]
    
    return board == str(Board.parse(board))

print(all([test() for _ in range(100)]))

True


### Tests misceláneos
Ejemplos del correcto funcionamiento de la comprobación de resolución, comprobación de direcciones y modificación del estado.

In [3]:
Board.parse("#####\n#* *#\n#####").is_solved()

b1 = Board.parse("###\n# #\n#@#\n###")
print(b1.can_move_towards("N"))
print(b1.can_move_towards("E"))
print(b1.can_move_towards("S"))
print(b1.can_move_towards("W"))
print()

b2 = Board.parse("###\n#.#\n#$#\n#@#\n###")
print(b2)
print(b2.is_solved())
print()
b2.move_towards("N")
print(b2)
print(b2.is_solved())

True
False
False
False

###
#.#
#$#
#@#
###
False

###
#*#
#@#
# #
###
True


### Cómputo de casillas muertas
Para representar casillas muertas se ha utilizado el símbolo `_` ya que en el formato XSB a veces se utiliza para representar casillas fuera del mapa, que siempre están muertas. El símbolo para el juegador cuando está en una de estas casillas es `&`, esta selección es completamente arbitraria; y para una caja, `!`, como advertencia.

In [4]:
print(Board.parse("""    #####
    #   #
    #$  #
  ###  $##
  #  $ $ #
### # ## #   ######
#   # ## #####  ..#
# $  $          ..#
##### ### #@##  ..#
    #     #########
    ####### """).compute_dead())

____#####
____#___#
____#$  #
__###  $##
__#_ $ $_#
### # ## #___######
#_  # ## #####_ ..#
#_$  $          ..#
##### ### #&##_ ..#
____#_____#########
____#######_


In [5]:
from search import *

### Problema
Gracias a la clase auxiliar, la representación del problema es trivial. Lo único notable es el cómputo de las casillas muertas en la inicialización del problema y la comprobación de que el problema es irresoluble en la generación de las acciones.

El estado se representa con la forma textual XSB y las acciones, con la abreviatura de la dirección cardinal en la que queremos mover al jugador.

In [6]:
class Sokoban(Problem):
    def __init__(self, initial):
        self.initial = str(Board.parse(initial).compute_dead())
    
    def actions(self, state):
        state = Board.parse(state)
        
        if state.is_deadlocked():
            return []
        
        return filter(lambda d: state.can_move_towards(d), "NESW")
    
    def result(self, state, action):
        return str(Board.parse(state).move_towards(action))
    
    def goal_test(self, state):
        return Board.parse(state).is_solved()

### Heurística
Como heurística hemos utilizado la suma de distancias Manhattan mínimas dada su simpleza de cáculo y garantía de ser el límite inferior.  

Esta heurística se podría mejorar usando la suma mínima de distancias a metas *distintas* o calculando la distancia del camino de cada caja a la meta más cercana teniendo en cuenta los obstáculos. La primera opción tiene un coste computacional grande y debe ser calculada dinámicamente, mientras que la segunda se podría precalcular si no se tienen en cuenta las otras cajas como obstaculos. La implementación de estas heurísticas más avanzadas está fuera del alcance de este ejercicio.

In [7]:
from utils import memoize

def manhattan(a, b):
    return abs(a[0] - b[0]) + abs(a[1] - b[1])

manhattan_m = memoize(manhattan)

def manhattan_sokoban(node):
    state = Board.parse(node.state)
    
    return sum(map(lambda b: min(map(lambda g: manhattan_m(b, g), state.goals)), state.boxes))

### Ejemplo de ejecución
A continuación se resuelve el ejemplo del enunciado y se muestra la traza de estados y acciones de la solución.

In [8]:
%%timeit -r 1 -n 1

p = Sokoban("#####\n# ..####\n# $    #\n#  #$# #\n# @ .$ #\n########")

n = astar_search(p, manhattan_sokoban)
for m in n.path():
    print(m.action)
    print(m.state)
    print()

None
#####
#_..####
#_$   _#
#_ #$#_#
#_@ .$_#
########

W
#####
#_..####
#_$   _#
#_ #$#_#
#&  .$_#
########

N
#####
#_..####
#_$   _#
#& #$#_#
#_  .$_#
########

N
#####
#_..####
#&$   _#
#_ #$#_#
#_  .$_#
########

N
#####
#&..####
#_$   _#
#_ #$#_#
#_  .$_#
########

E
#####
#_+.####
#_$   _#
#_ #$#_#
#_  .$_#
########

S
#####
#_..####
#_@   _#
#_$#$#_#
#_  .$_#
########

E
#####
#_..####
#_ @  _#
#_$#$#_#
#_  .$_#
########

E
#####
#_..####
#_  @ _#
#_$#$#_#
#_  .$_#
########

E
#####
#_..####
#_   @_#
#_$#$#_#
#_  .$_#
########

E
#####
#_..####
#_    &#
#_$#$#_#
#_  .$_#
########

S
#####
#_..####
#_    _#
#_$#$#&#
#_  .$_#
########

S
#####
#_..####
#_    _#
#_$#$#_#
#_  .$&#
########

W
#####
#_..####
#_    _#
#_$#$#_#
#_  *@_#
########

W
#####
#_..####
#_    _#
#_$#$#_#
#_ $+ _#
########

N
#####
#_..####
#_  $ _#
#_$#@#_#
#_ $. _#
########

S
#####
#_..####
#_  $ _#
#_$# #_#
#_ $+ _#
########

E
#####
#_..####
#_  $ _#
#_$# #_#
#_ $.@_#
########

E
#####
#_..####
#_  $ _#

### Más problemas
Importamos problemas en formato XSB. Solo usamos los 49 primeros ya que a partir de ahí son mucho más complejos.

In [9]:
import urllib.request

contents = (urllib.request.urlopen("http://kantorek.webzdarma.cz/DOWNLOAD/duthen.txt")
            .read()
            .decode("utf-8", "ignore"))

boards = []
curr_board = ""
for line in contents.replace("\r", "").split("\n"):
    if line == "" or ";" in line or "'" in line or line.isspace():
        if curr_board is not "":
            boards.append(curr_board[:-1])
            curr_board = ""
    else:
        curr_board += line + "\n"

boards = boards[:49]

### Comparativa de algoritmos de búsqueda
Se comparan tres algoritmos de búsqueda, dos no informados: en anchura y coste uniforme; y uno informado: A* con heurística Manhattan.  

En la tabla se pueden ver los tiempos de ejecución para resolver los 49 tableros importados:  

| En anchura | Coste uniforme | A* |
| :-: | :-: | :-: |
| 4.27s | 6.96s | 5.67s |

Como se puede apreciar, el cáculo de la heurística no merece la pena porque no prioriza lo suficiente los movimientos con mayor promesa, probablemente debido a que muchas soluciones requieren alejar las cajas de las metas para acercarse al estado final.  
En este caso la búsqueda no informada es mejor porque no tenemos suficiente conocimiento de dominio para hacer una búsqueda informada.

In [ ]:
%timeit list(map(lambda x: breadth_first_graph_search(Sokoban(x)), boards))
%timeit list(map(lambda x: uniform_cost_search(Sokoban(x)), boards))
%timeit list(map(lambda x: astar_search(Sokoban(x), manhattan_sokoban), boards))

3.76 s ± 47.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
5.98 s ± 76.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### Soluciones
Finalmente se muestran las soluciones a los 49 tableros importados con los tiempos de ejecución de cada uno.

In [ ]:
for i, b in enumerate(boards):
    p = Sokoban(b)
    print(str(i + 1) + "/" + str(len(boards)))
    print(p.initial)
    %timeit -r 1 -n 1 print(breadth_first_graph_search(p).solution())
    print()

## Conclusión
Sokoban es difícil.  

Obtener una heurística viable requiere una gran cantidad de conocimiento de dominio y la resolución de tableros complejos requiere razonamiento abstracto para crear una estrategia que no implique en vez de explorar una gran parte del enorme espacio de estados.